<a href="https://colab.research.google.com/github/restrepo/medicion/blob/master/cienciometria/Query_CTR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# WOS+SCI+SCP+PTJ+CTR QUERY

Queries for the bibliographic datasets for 
* Web of Science (WOS), 
* Scielo (SCI)
* Scopus  (SCP)
* Puntaje (UDEA)
* Center (CTR)
of the scientific articles of Universidad de Antioquia. 

They were created with:

[WOS_SCI_SCP_PTJ_CTR.ipynb](./WOS_SCI_SCP_PTJ_CTR.ipynb)

In [2]:
import os
VERSION='NEW'
if os.getcwd()=='/content':
    !pip install openpyxl xlrd wosplus > /dev/null

## functions

In [7]:
import pandas as pd
import wosplus as wp
pd.set_option('display.max_colwidth',200)
from venn import draw_venn, generate_colors
import numpy as np
import fuzzywuzzy.process as fwp
from fuzzywuzzy import fuzz

##  Configure public links of  files in Google Drive
* If it is a Google Spreadsheet the corresponding file is downloaded as CSV
* If it is in excel or text file the file is downloaded  directly

To define your  own labeled IDs for public google drive files edit the next cell:

In [4]:
%%writefile drive.cfg
[FILES]
WOS_SCI_SCP_PTJ_CTR.json.gz=19E1C1kRk4I0V3uXojqko8-NEicWaPp1j
WOS_SCP_UDEA_SJR_SIU.xlsx=0BxoOXsn2EUNIQ3R4WDhvSzVLQ2s
Base_de_datos_investigadores_Definitiva.csv=12oalgUeKhpvzkTPBP8pXCeHTrF-KO223dy9ov9w9QKs
UDEA_authors_with_WOS_info.json=1o1eVT4JD0FMMICq_oxrTJOzWh47veBMw
produccion_fecha_vig_2003_2018.xlsx=1WbtX4K__TTLxXRjuLvqUYz9tuHCIlS5v
UDEA_WOS_SCI_SCP_PTJ.json=1OkVytKbxJwGvXZDkynkSoUDtkUOTaT4A

Overwriting drive.cfg


##  Load data bases

In [5]:
affil='Univ Antioquia'
drive_files=wp.wosplus('drive.cfg')

#### DEBUG: if False stop in UDEA_PTJ!!!!

if os.path.exists(UDEAjsonfile):
    UDEA=               pd.read_json(UDEAjsonfile,compression='gzip').reset_index(drop=True)
else:    
    UDEA=drive_files.read_drive_json(UDEAjsonfile,compression='gzip').reset_index(drop=True)

In [7]:
UDEAjsonfile='WOS_SCI_SCP_PTJ_CTR.json.gz'
UDEAjsonfile='UDEAtmp.json.gz'
tmp=drive_files.load_biblio(UDEAjsonfile,compression='gzip')# TODO CHANGE FOR LAST VERSION IN GOOGLE DRIVE
UDEA=drive_files.biblio['WOS'].copy().reset_index(drop=True)

In [8]:
from check_quality import *
check_quality(UDEA)

authors_WOS
13645
Leptonic charged Higgs decays in the Zee model ; authors_WOS: [] ; AU: Sierra, DA
Restrepo, D

Tipo contains UDEA
2240
UDEA_authors
10956
UDEA_authors → full_names (Extrapolado puntaje)
10956
UDEA_authors → "NOMBRE COMPLETO" (Extrapolado CENTRO)
7329
TI: "The inert doublet model" check WOS_author vs UDEA_authors
...


## Indices

See: 

`{'CÉDULA': 63508258.0,
  'DEPARTAMENTO': 'Instituto de Biología',
  'FACULTAD': 'Facultad de Ciencias Exactas y Naturales',
  'GRUPO': 'Sin Grupo Asociado',
  'INICIALES': 'I.',
  'NOMBRE COMPLETO': 'Idalyd Fonseca Gonzalez',
  'NOMBRES': 'Idalyd',
  'PRIMER APELLIDO': 'Fonseca',
  'SEGUNDO APELLIDO': 'Gonzalez',
  'WOS_affiliation': ['Univ Antioquia, Colombia.'],
  'WOS_author': ['FONSECA, IDALYD',
   'FONSECA-GONZALEZ, IDALYD',
   'Fonseca-Gonzalez, Idalyd',
   'Fonseca-Gonzalez, I.'],
  'full_name': 'FONSECA GONZALEZ IDALYD'}`

In [9]:
#TODO: Improve indices
def get_groups(l,g):
    for d in l:
        gt=d.get('GRUPO')
        if gt and type( gt )==str:
            gs=gt.replace(
                ', Grupo','; Grupo'
            ).split('; ')
            for gg in gs:
                if gg not in g:
                    g.append(gg)
    return g


json_column='UDEA_authors'
facultades={'key':'FACULTAD',
            'values' : UDEA.UDEA_authors.apply(lambda l: 
                         [d.get('FACULTAD') for d in l] 
                          if type(l)==list else None
                          ).dropna().apply(pd.Series).stack().unique()}
departamentos={'key':'DEPARTAMENTO',
            'values' :UDEA.UDEA_authors.apply(lambda l: 
                         [d.get('DEPARTAMENTO') for d in l] 
                          if type(l)==list else None
                        ).dropna().apply(pd.Series).stack().unique()}
g=[]
#append to g
tmp=UDEA.UDEA_authors.apply(lambda l: 
                        get_groups(l,g)
        if type(l)==list else None
                        )
grupos={'key':'GRUPO',
            'values' :g}
nombre_completo={'key'    : 'NOMBRE COMPLETO',
                 'values' : UDEA[json_column].apply(lambda l: 
                            [d.get('NOMBRE COMPLETO') for d in l] 
                            if type(l)==list else None
                            ).dropna().apply(pd.Series).stack().unique()
                }#Nombres Apellidos. #TODO: Be sure to include full list
full_name={'key'    : 'full_name',
                 'values' : UDEA[json_column].apply(lambda l: 
                            [d.get('full_name') for d in l] 
                            if type(l)==list else None
                            ).dropna().apply(pd.Series).stack().unique()
                }#Apellidos Nombres . #TODO: Be sure to include full list

In [20]:
fwp.extractOne('Grupo de Fenomenología de Interacciones Fundamentales',g,scorer=fuzz.partial_token_sort_ratio)

('Grupo de Fenomenologia de Interacciones Fundamentales', 98)

In [192]:
nombre_completo['values'].shape,full_name['values'].shape

((649,), (1727,))

## Query function

In [58]:
def query_json_column(q,df=UDEA,json_column='UDEA_authors',
                        choices=nombre_completo,scorer=fuzz.partial_token_sort_ratio,**kwargs):
    fchoices=fwp.extractOne(q,choices['values'],scorer=scorer)[0]
    dfF=df[df[json_column].apply(lambda l: True in [ d.get(choices['key'])==fchoices
                                    for d in l] if type(l)==list else False)]
    return dfF.reset_index(drop=True)

In [59]:
r=query_json_column('Diego Alejandro Restrepo Quintero',df=UDEA,json_column='UDEA_authors',
                        choices=nombre_completo,scorer=fuzz.partial_token_sort_ratio,score_cutoff=79)

In [60]:
r.shape

(36, 181)

In [61]:
r[['TI','AU','authors_WOS',json_column]]

,TI,AU,authors_WOS,UDEA_authors
0,The inert doublet model,"Arias, C\nMartins, J\nMartinez, H\nRon, E\nSalzmann, C\nVasconcelos, GMS\nVillalba, F\n","[{'WOS_author': 'Arias, C.', 'i': 0, 'affiliation': ['Univ Antioquia, Inst Fis, AA 1226, Medellin, Colombia.']}]","[{'CÉDULA': 15386534.0, 'NOMBRE COMPLETO': 'Oscar Alberto Zapata Noreña', 'full_name': 'ZAPATA NOREÑA OSCAR ALBERTO', 'NOMBRES': 'Oscar Alberto', 'PRIMER APELLIDO': 'Zapata', 'DEPARTAMENTO': 'Inst..."
1,Leptonic charged Higgs decays in the Zee model,"Sierra, DA\nRestrepo, D\n",[],"[{'CÉDULA': 98554575.0, 'NOMBRE COMPLETO': 'Diego Alejandro Restrepo Quintero', 'full_name': 'RESTREPO QUINTERO DIEGO ALEJANDRO', 'NOMBRES': 'Diego Alejandro', 'PRIMER APELLIDO': 'Restrepo', 'DEPA..."
2,Collider signals of gravitino dark matter in bilinearly broken R-parity,"Hirsch, M\nPorod, W\nResterpo, D\n",[],"[{'CÉDULA': 98554575.0, 'NOMBRE COMPLETO': 'Diego Alejandro Restrepo Quintero', 'full_name': 'RESTREPO QUINTERO DIEGO ALEJANDRO', 'NOMBRES': 'Diego Alejandro', 'PRIMER APELLIDO': 'Restrepo', 'DEPA..."
3,Probing neutrino mass with multilepton production at the Tevatron in the simplest R-parity violation model,"Magro, MB\nde Campos, F\nEboli, OJP\nPorod, W\nRestrepo, D\nValle, JWF\n",[],"[{'CÉDULA': 98554575.0, 'NOMBRE COMPLETO': 'Diego Alejandro Restrepo Quintero', 'full_name': 'RESTREPO QUINTERO DIEGO ALEJANDRO', 'NOMBRES': 'Diego Alejandro', 'PRIMER APELLIDO': 'Restrepo', 'DEPA..."
4,Baryonic violation of R parity from anomalous U(1)(H),"Florez, A\nRestrepo, D\nVelasquez, M\nZapata, O\n","[{'WOS_author': 'Florez, Andres', 'i': 0, 'affiliation': ['Univ Antioquia, Inst Fis, Medellin 1226, Colombia.']}, {'WOS_author': 'Restrepo, Diego', 'i': 1, 'affiliation': ['Univ Antioquia, Inst Fi...","[{'CÉDULA': 98554575.0, 'NOMBRE COMPLETO': 'Diego Alejandro Restrepo Quintero', 'NOMBRES': 'Diego Alejandro', 'DEPARTAMENTO': 'Instituto de Física', 'GRUPO': 'Grupo de Fenomenologia de Interaccion..."
5,LSP sneutrino novel decays,"Sierra, DA\nRestrepo, D\nSpinner, S\n","[{'WOS_author': 'Restrepo, D.', 'i': 0, 'affiliation': ['Univ Antioquia, Inst Fis, Medellin 1226, Colombia.']}]","[{'CÉDULA': 98554575.0, 'NOMBRE COMPLETO': 'Diego Alejandro Restrepo Quintero', 'NOMBRES': 'Diego Alejandro', 'DEPARTAMENTO': 'Instituto de Física', 'GRUPO': 'Grupo de Fenomenologia de Interaccion..."
6,Scalar dark matter and fermion coannihilations in the radiative seesaw model,"Klasen, M\nYaguna, CE\nRuiz-Alvarez, JD\nRestrepo, D\nZapata, O\n","[{'WOS_author': 'Ruiz-Alvarez, Jose D.', 'i': 0, 'affiliation': ['Univ Antioquia, Inst Fis, Medellin 1226, Colombia.']}, {'WOS_author': 'Restrepo, Diego', 'i': 1, 'affiliation': ['Univ Antioquia, ...","[{'CÉDULA': 15386534.0, 'NOMBRE COMPLETO': 'Oscar Alberto Zapata Noreña', 'NOMBRES': 'Oscar Alberto', 'DEPARTAMENTO': 'Instituto de Física', 'GRUPO': 'Grupo de Fenomenologia de Interacciones Funda..."
7,"Connection of gamma rays, dark matter, and Higgs boson searches at the LHC","Ruiz-Alvarez, JD\nPires, CADS\nQueiroz, FS\nRestrepo, D\nda Silva, PSR\n","[{'WOS_author': 'Ruiz-Alvarez, J. D.', 'i': 0, 'affiliation': ['Univ Antioquia, Inst Fis, Medellin 1226, Colombia.']}, {'WOS_author': 'Restrepo, D.', 'i': 1, 'affiliation': ['Univ Antioquia, Inst ...","[{'CÉDULA': 98554575.0, 'NOMBRE COMPLETO': 'Diego Alejandro Restrepo Quintero', 'NOMBRES': 'Diego Alejandro', 'DEPARTAMENTO': 'Instituto de Física', 'GRUPO': 'Grupo de Fenomenologia de Interaccion..."
8,Probing neutralino properties in minimal supergravity with bilinear R-parity violation,"Campos, F\nEboli, OJP\nMagro, MB\nPorod, W\nRestrepo, D\nDas, SP\nHirsch, M\nValle, JWF\n","[{'WOS_author': 'Restrepo, D.', 'i': 0, 'affiliation': ['Univ Antioquia, Inst Fis, Medellin 1226, Colombia.']}]","[{'CÉDULA': 98554575.0, 'NOMBRE COMPLETO': 'Diego Alejandro Restrepo Quintero', 'NOMBRES': 'Diego Alejandro', 'DEPARTAMENTO': 'Instituto de Física', 'GRUPO': 'Grupo de Fenomenologia de Interaccion..."
9,Radiative neutrino masses in the singlet-do

In [62]:
r=query_json_column('RESTREPO QUINTERO DIEGO ALEJANDRO',df=UDEA,json_column='UDEA_authors',
                        choices=full_name,scorer=fuzz.partial_token_sort_ratio,score_cutoff=79)

In [63]:
r[['TI','AU','authors_WOS',json_column]]

,TI,AU,authors_WOS,UDEA_authors
0,The inert doublet model,"Arias, C\nMartins, J\nMartinez, H\nRon, E\nSalzmann, C\nVasconcelos, GMS\nVillalba, F\n","[{'WOS_author': 'Arias, C.', 'i': 0, 'affiliation': ['Univ Antioquia, Inst Fis, AA 1226, Medellin, Colombia.']}]","[{'CÉDULA': 15386534.0, 'NOMBRE COMPLETO': 'Oscar Alberto Zapata Noreña', 'full_name': 'ZAPATA NOREÑA OSCAR ALBERTO', 'NOMBRES': 'Oscar Alberto', 'PRIMER APELLIDO': 'Zapata', 'DEPARTAMENTO': 'Inst..."
1,Leptonic charged Higgs decays in the Zee model,"Sierra, DA\nRestrepo, D\n",[],"[{'CÉDULA': 98554575.0, 'NOMBRE COMPLETO': 'Diego Alejandro Restrepo Quintero', 'full_name': 'RESTREPO QUINTERO DIEGO ALEJANDRO', 'NOMBRES': 'Diego Alejandro', 'PRIMER APELLIDO': 'Restrepo', 'DEPA..."
2,Collider signals of gravitino dark matter in bilinearly broken R-parity,"Hirsch, M\nPorod, W\nResterpo, D\n",[],"[{'CÉDULA': 98554575.0, 'NOMBRE COMPLETO': 'Diego Alejandro Restrepo Quintero', 'full_name': 'RESTREPO QUINTERO DIEGO ALEJANDRO', 'NOMBRES': 'Diego Alejandro', 'PRIMER APELLIDO': 'Restrepo', 'DEPA..."
3,Probing neutrino mass with multilepton production at the Tevatron in the simplest R-parity violation model,"Magro, MB\nde Campos, F\nEboli, OJP\nPorod, W\nRestrepo, D\nValle, JWF\n",[],"[{'CÉDULA': 98554575.0, 'NOMBRE COMPLETO': 'Diego Alejandro Restrepo Quintero', 'full_name': 'RESTREPO QUINTERO DIEGO ALEJANDRO', 'NOMBRES': 'Diego Alejandro', 'PRIMER APELLIDO': 'Restrepo', 'DEPA..."
4,Baryonic violation of R parity from anomalous U(1)(H),"Florez, A\nRestrepo, D\nVelasquez, M\nZapata, O\n","[{'WOS_author': 'Florez, Andres', 'i': 0, 'affiliation': ['Univ Antioquia, Inst Fis, Medellin 1226, Colombia.']}, {'WOS_author': 'Restrepo, Diego', 'i': 1, 'affiliation': ['Univ Antioquia, Inst Fi...","[{'CÉDULA': 98554575.0, 'NOMBRE COMPLETO': 'Diego Alejandro Restrepo Quintero', 'NOMBRES': 'Diego Alejandro', 'DEPARTAMENTO': 'Instituto de Física', 'GRUPO': 'Grupo de Fenomenologia de Interaccion..."
5,LSP sneutrino novel decays,"Sierra, DA\nRestrepo, D\nSpinner, S\n","[{'WOS_author': 'Restrepo, D.', 'i': 0, 'affiliation': ['Univ Antioquia, Inst Fis, Medellin 1226, Colombia.']}]","[{'CÉDULA': 98554575.0, 'NOMBRE COMPLETO': 'Diego Alejandro Restrepo Quintero', 'NOMBRES': 'Diego Alejandro', 'DEPARTAMENTO': 'Instituto de Física', 'GRUPO': 'Grupo de Fenomenologia de Interaccion..."
6,Scalar dark matter and fermion coannihilations in the radiative seesaw model,"Klasen, M\nYaguna, CE\nRuiz-Alvarez, JD\nRestrepo, D\nZapata, O\n","[{'WOS_author': 'Ruiz-Alvarez, Jose D.', 'i': 0, 'affiliation': ['Univ Antioquia, Inst Fis, Medellin 1226, Colombia.']}, {'WOS_author': 'Restrepo, Diego', 'i': 1, 'affiliation': ['Univ Antioquia, ...","[{'CÉDULA': 15386534.0, 'NOMBRE COMPLETO': 'Oscar Alberto Zapata Noreña', 'NOMBRES': 'Oscar Alberto', 'DEPARTAMENTO': 'Instituto de Física', 'GRUPO': 'Grupo de Fenomenologia de Interacciones Funda..."
7,"Connection of gamma rays, dark matter, and Higgs boson searches at the LHC","Ruiz-Alvarez, JD\nPires, CADS\nQueiroz, FS\nRestrepo, D\nda Silva, PSR\n","[{'WOS_author': 'Ruiz-Alvarez, J. D.', 'i': 0, 'affiliation': ['Univ Antioquia, Inst Fis, Medellin 1226, Colombia.']}, {'WOS_author': 'Restrepo, D.', 'i': 1, 'affiliation': ['Univ Antioquia, Inst ...","[{'CÉDULA': 98554575.0, 'NOMBRE COMPLETO': 'Diego Alejandro Restrepo Quintero', 'NOMBRES': 'Diego Alejandro', 'DEPARTAMENTO': 'Instituto de Física', 'GRUPO': 'Grupo de Fenomenologia de Interaccion..."
8,Probing neutralino properties in minimal supergravity with bilinear R-parity violation,"Campos, F\nEboli, OJP\nMagro, MB\nPorod, W\nRestrepo, D\nDas, SP\nHirsch, M\nValle, JWF\n","[{'WOS_author': 'Restrepo, D.', 'i': 0, 'affiliation': ['Univ Antioquia, Inst Fis, Medellin 1226, Colombia.']}]","[{'CÉDULA': 98554575.0, 'NOMBRE COMPLETO': 'Diego Alejandro Restrepo Quintero', 'NOMBRES': 'Diego Alejandro', 'DEPARTAMENTO': 'Instituto de Física', 'GRUPO': 'Grupo de Fenomenologia de Interaccion..."
9,Radiative neutrino masses in the singlet-do

In [64]:
r=query_json_column('Instituto de Física',df=UDEA,json_column='UDEA_authors',
                        choices=departamentos,scorer=fuzz.partial_token_sort_ratio,score_cutoff=79)

In [65]:
r.shape

(849, 181)

In [174]:
def query_json_column(q,df=UDEA,json_column='UDEA_authors',
                        choices=nombre_completo,scorer=fuzz.partial_token_sort_ratio,**kwargs):
    fchoices=fwp.extractOne(q,choices['values'],scorer=scorer)[0]
    dfF=df[df[json_column].apply(lambda l: True in [ d.get(choices['key']).find(fchoices)>-1 
                                    for d in l if d.get(choices['key'])] if type(l)==list else False)]
    #dfF=df[df[json_column].apply(lambda l: [ d.get(choices['key']) 
    #                                for d in l if d.get(choices['key'])] 
    #                             ).str[0].str.contains(fchoices).fillna(False)]
    return dfF.reset_index(drop=True)

In [130]:
fchoices

'Grupo de Fenomenologia de Interacciones Fundamentales'

In [105]:
choices=grupos
scorer=fuzz.partial_token_sort_ratio
fchoices=fwp.extractOne(q,choices['values'],scorer=scorer)[0]


SyntaxError: unexpected EOF while parsing (<ipython-input-105-376829e9f4b8>, line 4)

In [147]:
fchoices=='Grupo de Fenomenologia de Interacciones Fundamentales'

True

In [173]:
fchoices='Grupo de Biofísica-UdeA'
UDEA[UDEA[json_column].apply(lambda l: [ d.get(choices['key']) for d in l if d.get(choices['key'])] 
                            ).str[0].str.contains(fchoices).fillna(False)][json_column].apply(lambda l: [ d.get(choices['key']) for d in l if d.get(choices['key'])] 
                            )

7477     [Grupo de Biofísica-UdeA, Grupo de Optica y Fotónica, Grupo de Optica y Fotónica, Grupo de Estado Sólido, Instrumentación Científica y Microelectrónica]
8355                                                                        [Grupo de Biofísica-UdeA, Grupo de Investigación en Fisiología y Bioquímica - Physis]
8713                                                                        [Grupo de Biofísica-UdeA, Grupo de Investigación en Fisiología y Bioquímica - Physis]
9724                                                                        [Grupo de Biofísica-UdeA, Grupo de Investigación en Fisiología y Bioquímica - Physis]
11957                                                                       [Grupo de Biofísica-UdeA, Grupo de Investigación en Fisiología y Bioquímica - Physis]
13067                                                                           [Grupo de Biofísica-UdeA, Grupo de Optica y Fotónica, Grupo de Optica y Fotónica]
15154                       

In [171]:
df=UDEA
df[df[json_column].apply(lambda l: True in [ d.get(choices['key']).find(fchoices)>-1 
                                    for d in l if d.get(choices['key'])] if type(l)==list else False)][json_column].apply(lambda l: [ d.get(choices['key']) for d in l if d.get(choices['key'])] 
                            )

7477     [Grupo de Biofísica-UdeA, Grupo de Optica y Fotónica, Grupo de Optica y Fotónica, Grupo de Estado Sólido, Instrumentación Científica y Microelectrónica]
8355                                                                        [Grupo de Biofísica-UdeA, Grupo de Investigación en Fisiología y Bioquímica - Physis]
8713                                                                        [Grupo de Biofísica-UdeA, Grupo de Investigación en Fisiología y Bioquímica - Physis]
9724                                                                        [Grupo de Biofísica-UdeA, Grupo de Investigación en Fisiología y Bioquímica - Physis]
11957                                                                       [Grupo de Biofísica-UdeA, Grupo de Investigación en Fisiología y Bioquímica - Physis]
12048           [Grupo de Investigación en Fisiología y Bioquímica - Physis, Grupo de Biofísica-UdeA, Grupo de Investigación en Fisiología y Bioquímica - Physis]
13067                       

In [68]:
r=query_json_column('Grupo de Fenomenología de Interacciones Fundamentales',df=UDEA,json_column='UDEA_authors',
                        choices=grupos,scorer=fuzz.partial_token_sort_ratio,score_cutoff=79)

Grupo de Fenomenologia de Interacciones Fundamentales


In [69]:
r.shape

(77, 181)

In [55]:
q='Grupo de Magnetismo y Simulación'
query_json_column(q,df=UDEA,json_column='UDEA_authors',choices=grupos
                        )

Grupo de Magnetismo y Simulación


,AB,AF,AR,AU,BA,BE,BF,BN,BP,C1,...,UDEA_tipo mat,UDEA_tipo mov,UDEA_título,UDEA_título_list,UDEA_valor item,UT,VL,WC,Z9,authors_WOS


In [87]:
q='Grupo de Magnetismo y Simulación'

'Grupo de Magnetismo y Simulación'

In [91]:
q='Grupo de Fenomenologia de Interacciones Fundamentales'

In [92]:
UDEA[UDEA[json_column].apply(lambda l: [d.get('GRUPO') if type( d.get('GRUPO'))==str 
                                   else None for  d in l ] if l else l ).astype(str).str.contains(q)].shape

(80, 181)

In [ ]:
gdf=pd.DataFrame()
for g in grupos['values']:
    r=UDEA[ UDEA.UDEA_authors.astype(str).str.lower().apply(unidecode.unidecode
          ).str.contains(unidecode.unidecode( g.lower() )
                        ) ]
    gdf=gdf.append( {'Group':g,'articles':r.shape[0]},ignore_index=True )
gdf['articles']=gdf['articles'].astype(int)

In [33]:
gdf.sort_values('articles',ascending=False).reset_index(drop=True)

,Group,articles
0,Sin Grupo Asociado,399
1,Grupo de Materia Condensada-UdeA,261
2,"Centro de Investigación, Innovación y Desarrollo de Materiales - CIDEMAT - Anteriormente: Grupo de Corrosión y Protección",231
3,"Centro de Investigación, Innovación y Desarrollo de Materiales - CIDEMAT - Anteriormente: Grupo de Corrosión y Protección,",231
4,"Grupo Reproducción, Inmunovirología, Infección y Cáncer",226
5,Inmunovirología,226
6,"Grupo de Manejo Eficiente de la Energía, GIMEL",213
7,"Grupo de Neurociencias de Antioquia, SINAPSIS",211
8,Grupo de Neurociencias de Antioquia,211
9,Programa de Estudio y Control de Enfermedades Tropicales,194


In [260]:
UDEA[ UDEA.UDEA_authors.astype(str).str.contains('Grupo de Magnetismo y Simulación') ].shape

(47, 181)

(82, 181)

In [235]:
import unidecode

In [236]:
gdf[gdf.Group.str.lower().apply(unidecode.unidecode).str.contains('feno')]

,Group,articles
143,Grupo de Fenomenologia de Interacciones Fundamentales,77


In [227]:
ss

In [228]:
ss.shape

(245,)

In [284]:
len(g)

249